In [24]:
import json
import pandas as pd
import random
import numpy as np

#Set seed
seed = 42

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)

set_seed(42)
data = [ "train", "test" ]

#Read jsons and combine to concatenated dataframe
cdf = pd.concat([pd.read_json( f"{d}.json" ) for d in data])
cdf.head()

,utterance,slots,intent
0,i want to fly from boston at 838 am and arrive...,O O O O O B-fromloc.city_name O B-depart_time....,flight
1,what flights are available from pittsburgh to ...,O O O O O B-fromloc.city_name O B-toloc.city_n...,flight
2,what is the arrival time in san francisco for ...,O O O B-flight_time I-flight_time O B-fromloc....,flight_time
3,cheapest airfare from tacoma to orlando,B-cost_relative O O B-fromloc.city_name O B-to...,airfare
4,round trip fares from pittsburgh to philadelph...,B-round_trip I-round_trip O O B-fromloc.city_n...,airfare


In [25]:
#drop 'slots'

def process( df ):
  df.drop('slots', axis=1, inplace=True)

  def correct_label( row ):
    return " ".join( [word.capitalize() for word in row.split("_")] ) + "."

  df['intent'] = df['intent'].apply( correct_label )
  df = df.rename( columns={ 'utterance': 'text', 'intent' : 'label'} )
  #reorder columns
  df = df[['label', 'text']]
  return df

cdf = process( cdf )
cdf.head()

,label,text
0,Flight,i want to fly from boston at 838 am and arrive...
1,Flight,what flights are available from pittsburgh to ...
2,Flight Time,what is the arrival time in san francisco for ...
3,Airfare,cheapest airfare from tacoma to orlando
4,Airfare,round trip fares from pittsburgh to philadelph...


In [26]:
cdf.to_csv( "data.csv", index=False )

In [27]:
from sklearn.model_selection import train_test_split
train_df = pd.read_json("train.json")
test_df = pd.read_json("test.json")

train_df = process( train_df )
test_df = process( test_df )

train_df.to_csv( "data/train.csv" , index=False)

#split test_df into test and validation
test_df, val_df = train_test_split( test_df, test_size=0.5, random_state=42 )

test_df.to_csv( "data/test.csv", index=False )
val_df.to_csv( "data/val.csv", index=False )